# Типовая задача A/B-тестирования. Сравнения нового дизайна лендинга со старым

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('ab_testing.csv')
df

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


## Изучение и подготовка данных

In [3]:
# проверяем типы данных и пропуски
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [4]:
# сколько всего зафиксированы визитов на сайт?
df.shape

(294478, 5)

In [5]:
# сколько уникальных пользователей?
df['user_id'].nunique()

290584

In [6]:
# какой процент пользователей по всем нашим данным совершил покупку?
df.converted.mean()

0.11965919355605512

Пользователей разделили на 2 группы. Первым показывали только старую версию лендинга, вторым – только новую.


In [7]:
# почему группа может не совпадать с версией лендинга?
df[['group', 'landing_page']].value_counts()

group      landing_page
treatment  new_page        145311
control    old_page        145274
treatment  old_page          1965
control    new_page          1928
dtype: int64

In [8]:
# уберем эти данные, как ошибочные. 
# Там где treatment != new_page будет control != old_page соответственно
df = df[((df['group'] == 'treatment') == (df['landing_page']== 'new_page')) == True]

In [9]:
# данные реально были ошибочными, уникальных пользователей осталось столько же  
df['user_id'].nunique()

290584

In [10]:
# одинаковых пользователей не осталось?
df[df.duplicated(['user_id'], keep=False)] 
# keep=False позволяет отобразить не только дубль, но и "оригинал"

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [11]:
# убираем (только дубль)
df = df.drop_duplicates(subset ='user_id', keep ='first')

In [12]:
# изменился ли процент конверсии после подготовки данных?
df.converted.mean()

0.11959708724499628

In [13]:
# какая конверсия на старой версии?
df[df.group == 'control']['converted'].mean()

0.1203863045004612

In [14]:
# а на новой?
df[df.group == 'treatment']['converted'].mean()

0.11880806551510564

Работа проделана зря??? Старый сайт лучше?
![image](https://www.meme-arsenal.com/memes/d036b99a114cfe7c34226ea5ca078462.jpg)

## Проверяем

Какой критерий нужно применить?

Составляем [матрицу сопряженности](http://www.machinelearning.ru/wiki/index.php?title=%D0%A2%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B0_%D1%81%D0%BE%D0%BF%D1%80%D1%8F%D0%B6%D0%B5%D0%BD%D0%BD%D0%BE%D1%81%D1%82%D0%B8)

In [19]:
import scipy.stats
control_group = df[df.group == 'control']
treatment_group = df[df.group == 'treatment']

control_converted = control_group.converted.sum()
treatment_converted = treatment_group.converted.sum()

control_notconverted = control_group.converted.count() - control_converted
treatment_notconverted = treatment_group.converted.count() - treatment_converted

cont_matrix = [[control_converted, control_notconverted], [treatment_converted, treatment_notconverted]]
cont_matrix

[[17489, 127785], [17264, 128046]]

Проверяем гипотезу, используя критерий [кси-квадрат](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).
Применяем [поправку Йетса](https://en.wikipedia.org/wiki/Yates%27s_correction_for_continuity) для размерности матрицы сопряженности 2х2.

In [22]:
res = scipy.stats.chi2_contingency(cont_matrix, correction=True)
res[1]

0.1918222809623566

In [23]:
alpha = 0.05
if res[1] <= alpha: 
    print('У лендингов разная конверсия') 
else: 
    print('Не можем с уверенностью отвергнуть нулевую гипотезу') 

Не можем с уверенностью отвергнуть нулевую гипотезу


Что делаем дальше?

Не стесняемся использовать [готовые инструменты](https://mindbox.ru/ab-test-calculator/).  
Напоминание про [статистическую мощность](https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D0%BC%D0%BE%D1%89%D0%BD%D0%BE%D1%81%D1%82%D1%8C).